In [7]:
import pandas as pd
import numpy as np
import datetime as dt
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels.api as sm
import pulp
import scipy
from scipy.optimize import minimize


import opendatasets as od
from IPython.core.display import display, HTML
display(HTML("<style>.container { width : 98% !important; }</style>"))

pd.set_option('display.max_columns',500)

In [3]:
# Import all classes of PuLP module
from pulp import *

# Create the problem variable to contain the problem data
model = LpProblem("ProductionProblem", LpMaximize)

# Create 3 variables tables, chairs, and bookcases

foldy = LpVariable("foldy", 0, None, LpContinuous) # takes 1.5 hrs to make a foldy phone
tiny = LpVariable("tiny", 0, None,  LpContinuous)  # takes 2 hrs to make a tiny phone

# Create maximize objective function
model += 900 * foldy + 1100 * tiny

# Create the constraints
model += 2 * tiny + 1.5 * foldy <= 2999.5, "labor"
model += tiny >= 200, "tinyphone_min"
model += foldy >= 500, "foldyphone_min"

# The problem is solved using PuLP's choice of Solver
model.solve()

# Each of the variables is printed with it's resolved optimum value
for v in model.variables():
    print(v.name, "=", v.varValue)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/APinkerton/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0g/4l0cmb9j43jgr0cxr8nx6sdm0000gn/T/bf68ddb0be004e239f6118cd2eaefa67-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/0g/4l0cmb9j43jgr0cxr8nx6sdm0000gn/T/bf68ddb0be004e239f6118cd2eaefa67-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 15 RHS
At line 19 BOUNDS
At line 20 ENDATA
Problem MODEL has 3 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-2) columns and 0 (-4) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 1779700
After Postsolve, objective 1779700, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1779700 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions

In [4]:
# Import all classes of PuLP module
from pulp import *

# Create the problem variable to contain the problem data
model = LpProblem("ProductionProblem", LpMaximize)

# Create 3 variables tables, chairs, and bookcases

foldy = LpVariable("foldy", 0, None, LpContinuous) # takes 1.5 hrs to make a foldy phone
tiny = LpVariable("tiny", 0, None,  LpContinuous)  # takes 2 hrs to make a tiny phone

# Create maximize objective function
model += 900 * foldy + 1100 * tiny

# Create the constraints
model += 2 * tiny + 1.5 * foldy <= 2999.5, "labor"
model += tiny >= 200, "tinyphone_min"
model += foldy >= 500, "foldyphone_min"

# The problem is solved using PuLP's choice of Solver
model.solve()

# Each of the variables is printed with it's resolved optimum value
for v in model.variables():
    print(v.name, "=", v.varValue)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/APinkerton/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0g/4l0cmb9j43jgr0cxr8nx6sdm0000gn/T/4ac933f3332140a48e0228af395c9934-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/0g/4l0cmb9j43jgr0cxr8nx6sdm0000gn/T/4ac933f3332140a48e0228af395c9934-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 15 RHS
At line 19 BOUNDS
At line 20 ENDATA
Problem MODEL has 3 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-3) rows, 0 (-2) columns and 0 (-4) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 1779700
After Postsolve, objective 1779700, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1779700 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions

In [19]:
# CAN USE THIS ONE BELOW, BUT THE LOOPS MAKE IT HARDER TO DISSECT

In [5]:
from pulp import *
def define_problem_objective(steak_price: float, peanut_butter_price: float):
    problem = LpProblem(name="DietProblem", sense=LpMinimize)
    S = LpVariable(name = "Steak_In_Pounds", lowBound = 0, cat = LpContinuous)
    P = LpVariable(name = "Peanut_Butter_In_Pounds", lowBound = 0, cat = LpContinuous)
    problem += (steak_price*S) + (peanut_butter_price * P), "MinimizeCost"
    return (problem, S, P)

def add_constraints(problem: LpProblem, S: LpVariable, P: LpVariable):
    problem += (2*S) + P >= 4, "MinimumProteinIntake"
    problem += S >= 0, "MinimumSteakQuantity"
    problem += P >= 0, "MinimumPeanutButterQuantity"
    
prices = [(3,2), (4,2), (4,2.5), (12, 2), (10,3)]
for steak_price, peanut_butter_price in prices:
    print("\n****************************************************")
    print(f"Steak Price: {steak_price} | Peanut Butter Price: {peanut_butter_price}")
    problem, steak, peanut_butter = define_problem_objective(steak_price, peanut_butter_price)
    add_constraints(problem, steak, peanut_butter)
    problem.solve(PULP_CBC_CMD(msg=0))
    print(f"Status of solution is: {LpStatus[problem.status]}")
    print(f"Optimal cost per day is: ${problem.objective.value()}")
    print(f"Steak (in pounds): {steak.value()}")
    print(f"Peanut Butter (in pounds): {peanut_butter.value()}")


****************************************************
Steak Price: 3 | Peanut Butter Price: 2
Status of solution is: Optimal
Optimal cost per day is: $6.0
Steak (in pounds): 2.0
Peanut Butter (in pounds): 0.0

****************************************************
Steak Price: 4 | Peanut Butter Price: 2
Status of solution is: Optimal
Optimal cost per day is: $8.0
Steak (in pounds): 2.0
Peanut Butter (in pounds): 0.0

****************************************************
Steak Price: 4 | Peanut Butter Price: 2.5
Status of solution is: Optimal
Optimal cost per day is: $8.0
Steak (in pounds): 2.0
Peanut Butter (in pounds): 0.0

****************************************************
Steak Price: 12 | Peanut Butter Price: 2
Status of solution is: Optimal
Optimal cost per day is: $8.0
Steak (in pounds): 0.0
Peanut Butter (in pounds): 4.0

****************************************************
Steak Price: 10 | Peanut Butter Price: 3
Status of solution is: Optimal
Optimal cost per day is: $12.0
Stea

In [8]:
# Simplest example without constraints or multiple variables
def f(x):
    y = (x**2) - (12 * x) + 20
    return y

x_start = 2.0

result = scipy.optimize.minimize(f,x_start)
# Can also do the below to watch it optimize:
# result = scipy.optimize.minimize(f, x_start, options = {"disp":True})

if result.success:
    print(f"x={result.x} y = {result.fun}")
else:
    print('Could not optimize')

x=[6.00000025] y = -15.999999999999936


In [9]:
# multiple variables, bounds, constraints
def f(xy):
    x = xy[0]
    y = xy[1]
    area = x * y
    return -area

xy_start = [50,50]

cons = ({'type':'eq','fun':lambda xy : (2*xy[0] + xy[1] - 100)})

bnds = ((1,100),(1,100))

result = scipy.optimize.minimize(f, x_start, constraints=cons, bounds=bnds)
# Can also do the below to watch it optimize:
# result = scipy.optimize.minimize(f, x_start, options = {"disp":True})

if result.success:
    xy = result.x
    x = xy[0]
    y = xy[1]
    print(f"x = {x} y = {y}")
else:
    print('Could not optimize')

x = 25.000005089367114 y = 49.99998982126577


In [10]:
# Optimizing cost for a variety of different components of a good, while still hitting nutritional mins / max's of certain nutrients
cost = pd.read_excel('/Users/APinkerton/Downloads/raw-materials-main/Costs.xlsx')
facts = pd.read_excel('/Users/APinkerton/Downloads/raw-materials-main/Nutrition Facts.xlsx',index_col=1,skiprows=1)
facts = facts[facts.columns[~facts.columns.str.match('Unnamed')]].copy()

In [11]:
cost['Ingredients'] = cost['Ingredients'].str.replace(' ','_')
facts.index = facts.index.str.replace(' ','_')

In [12]:
dict_costs = dict(zip(cost['Ingredients'],cost['Costs']))
cost

,Ingredients,Costs
0,Chicken,0.095
1,Beef,0.150
2,Mutton,0.100
3,Rice,0.002
4,Wheat_bran,0.005
5,Corn,0.012
6,Peanuts,0.013


In [13]:
variables = ['Chicken', 'Beef', 'Mutton', 'Rice', 'Wheat_bran', 'Corn', 'Peanuts']
facts

,Protein,Fat,Fibre,Salt,Sugar
Ingredients,,,,,
Chicken,0.100000,0.080000,0.001000,0.002,0.000
Beef,0.200000,0.100000,0.005000,0.005,0.000
Mutton,0.150000,0.110000,0.003000,0.007,0.000
Rice,0.000000,0.010000,0.100000,0.002,0.000
Wheat_bran,0.040000,0.010000,0.150000,0.008,0.000
Corn,0.032927,0.012805,0.028049,0.000,0.045
Peanuts,0.258000,0.492000,0.085000,0.001,0.047


In [14]:
model = LpProblem('Optimize Protein Bar',LpMinimize)
# Decision Variables:
x = LpVariable.dicts('Qty',[j for j in variables],lowBound=0,upBound=None,cat='continuous')
# Objective function
model += (lpSum([dict_costs[i]*x[i] for i in variables]))
# Constraints
model += (lpSum([x[i] for i in variables])) == 100 # total weight = 120 grams
model += (lpSum([x[i] * facts.loc[i,'Protein'] for i in variables])) >=22 # Protein at least 30 g
model += (lpSum([x[i] * facts.loc[i,'Fat'] for i in variables])) <=22 # Fat less than or eq to 22 g
model += (lpSum([x[i] * facts.loc[i, 'Fibre'] for i in variables])) >= 6 # Fibre at least 6 g
model += (lpSum([x[i] * facts.loc[i,'Salt'] for i in variables])) <=3 # Salt less than or eq to 3 g
model += (lpSum([x[i] * facts.loc[i,'Sugar'] for i in variables])) <=20 # Sugar less than or eq to 20 g

/Users/APinkerton/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [15]:
# Now solve / print results
model.solve()
print('Cost per bar = {:,} $'.format(round(value(model.objective),2)))
print('\n'+'Status:{}'.format(LpStatus[model.status]))
for v in model.variables():
    print(v.name,'=',round(v.varValue,2),'g')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/APinkerton/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0g/4l0cmb9j43jgr0cxr8nx6sdm0000gn/T/c4d8fb4c0d974365b7152dc5b4050573-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/0g/4l0cmb9j43jgr0cxr8nx6sdm0000gn/T/c4d8fb4c0d974365b7152dc5b4050573-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 11 COLUMNS
At line 54 RHS
At line 61 BOUNDS
At line 62 ENDATA
Problem MODEL has 6 rows, 7 columns and 35 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (-2) rows, 7 (0) columns and 27 (-8) elements
0  Obj 0 Primal inf 621.87976 (3)
7  Obj 11.076174 Primal inf 173.17622 (2)
Primal infeasible - objective value 11.076174
Presolved problem not optimal, resolve after postsolve
After Postsolve, objective 11.076174, infeasibilities - dual 0 (0), primal 4.831

In [16]:
import pandas as pd
from pulp import *

# Import Manufacturing Costs
manvar_costs = pd.read_excel('/Users/APinkerton/ipynbs/supply-chain-opt-main/variable_costs.xlsx', index_col = 0)
# Import Freight Costs
freight_costs = pd.read_excel('/Users/APinkerton/ipynbs/supply-chain-opt-main/freight_costs.xlsx', index_col = 0)
# Variable Cost
var_cost = freight_costs/1000 + manvar_costs 
# Import Plant Fixed Costs
fixed_costs = pd.read_excel('/Users/APinkerton/ipynbs/supply-chain-opt-main/fixed_cost.xlsx', index_col = 0)
# Import Low Capacity and High Capacity Plant
cap = pd.read_excel('/Users/APinkerton/ipynbs/supply-chain-opt-main/capacity.xlsx', index_col = 0)
# Import Demand
demand = pd.read_excel('/Users/APinkerton/ipynbs/supply-chain-opt-main/demand.xlsx', index_col=1,skiprows=1)
demand = demand[['Demand']].copy()


# Define Decision Variables
loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
size = ['Low', 'High']

# Initialize Class
model = LpProblem("Supply Chain Optimization", LpMinimize)


# Create Decision Variables
x = LpVariable.dicts("production_", [(i,j) for i in loc for j in loc],
                     lowBound=0, upBound=None, cat='continuous')
y = LpVariable.dicts("plant_", 
                     [(i,s) for s in size for i in loc], cat='Binary')

/Users/APinkerton/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [17]:
# Define Objective Function
model += (lpSum([fixed_costs.loc[i,s] * y[(i,s)] * 1000 for s in size for i in loc])
          + lpSum([var_cost.loc[i,j] * x[(i,j)]   for i in loc for j in loc]))

# Add Constraints
for j in loc:
    model += lpSum([x[(i, j)] for i in loc]) == demand.loc[j,'Demand']
for i in loc:
    model += lpSum([x[(i, j)] for j in loc]) <= lpSum([cap.loc[i,s]*y[(i,s)] * 1000
                                                       for s in size])

In [18]:
# Solve Model
model.solve()
print("Total Costs = {:,} ($/Month)".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))


# Dictionnary
dict_plant = {}
dict_prod = {}
for v in model.variables():
    if 'plant' in v.name:
        name = v.name.replace('plant__', '').replace('_', '')
        dict_plant[name] = int(v.varValue)
        p_name = name
    else:
        name = v.name.replace('production__', '').replace('_', '')
        dict_prod[name] = v.varValue
    print(name, "=", v.varValue)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/APinkerton/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0g/4l0cmb9j43jgr0cxr8nx6sdm0000gn/T/d4fbab6019a14c08b7a3255ec2cf6481-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/0g/4l0cmb9j43jgr0cxr8nx6sdm0000gn/T/d4fbab6019a14c08b7a3255ec2cf6481-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 131 RHS
At line 142 BOUNDS
At line 153 ENDATA
Problem MODEL has 10 rows, 35 columns and 60 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 8.7227e+07 - 0.00 seconds
Cgl0004I processed model has 10 rows, 35 columns (10 integer (10 of which binary)) and 60 elements
Cbc0038I Initial state - 3 integers unsatisfied sum - 0.603333
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 9.3579e+07 iterations 5
Cbc0038I Solution fo